## Experimental Setting 2: Features + rbf SVM 
### Task 2: Classification Claims vs Premises

**Importing necessary packages**

In [1]:
import pandas as pd
import numpy as np
import string
import collections
import spacy
nlp = spacy.load('en_core_web_sm')
from tensorflow.keras.utils import to_categorical
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize, RegexpParser

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/mariap/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mariap/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
filename = '../../../data/sentence_db_candidate.csv'
df = pd.read_csv(filename)

**Simple preprocessing of sentences with lowercasing and punctuation removal**

In [3]:
def preproc(sentence):
    sentence = sentence.lower()
    sentence = ''.join([i for i in sentence if i not in string.punctuation])
    return sentence

In [4]:
df['Speech'] = df['Speech'].apply(preproc)

**Below we are turning labels "Claim" and "Premise" into machine-readable classes: 1 and 0 respectively**

In [5]:
comps = ['Claim', 'Premise']
df = df.loc[(df['Component'].isin(comps))]

In [6]:
classes = []

for s in df.Component:
    if s == 'Claim':
        classes.append(1.0)
    else:
        classes.append(0.0)

In [7]:
df['Annotation'] = classes

In [8]:
df.shape

(22280, 19)

In [9]:
df = df[['Speech', 'Annotation', 'Set']]

In [10]:
df.head()

,Speech,Annotation,Set
3,and after 911 it became clear that we had to d...,1.0,TRAIN
4,and we also then finally had to stand up democ...,0.0,TRAIN
9,what we did in iraq was exactly the right thin...,1.0,TRAIN
10,if i had it to recommend all over again i woul...,0.0,TRAIN
11,the world is far safer today because saddam hu...,0.0,TRAIN


In [12]:
df['Annotation'].value_counts()

1.0    11964
0.0    10316
Name: Annotation, dtype: int64

### Part 1: Feature Engineering

**Now we are starting feature engineering. The functions are designed to take the full dataframe and a column with sentences and to output a dataframe with a newly added feature(s). Please refer to `linguistic_features.py` file to see/copy the documented feature functions**

### *Feature: Part-of-Speech (adverbs and adjectives)*

In [13]:
def list_count(lst):
    
    """
    :function: count the elements of a list -- the number of words with a respective POS or NER labels in a sentence. 
    :input: lst: list of tuples, where tuple has two elements -- a word and its POS or NER label
    :return: lst_count: list of dictionaries, where
    the dictionary consists of keys -- the elements are words and their POS or NER labels
    and values -- how many times each word and its POS or NER label occurs
    If a sentence has no POS or NER labels, return an empty list 

    """
    
    dic_counter = collections.Counter()
    
    for x in lst:
        dic_counter[x] += 1
    
    dic_counter = collections.OrderedDict( 
                     sorted(dic_counter.items(), 
                     key=lambda x: x[1], reverse=True))
    
    lst_count = [{key:value} for key,value in dic_counter.items()]
    
    return lst_count

In [14]:
def column_tag(lst_dics_tuples, tag):
    
    """
    :function: new column for each POS or NER tag category 
    :input: lst_dics_tuples: list of dictionaries with tuples 
            tag: POS or NER label from a list
    :return: tag: new column for each POS or NER label with their counts

    """
    
    if len(lst_dics_tuples) > 0:
        tag_type = []
        
        for dic_tuples in lst_dics_tuples:
            for tuple in dic_tuples:
                type, n = tuple[1], dic_tuples[tuple]
                tag_type = tag_type + [type]*n
                dic_counter = collections.Counter()
                for x in tag_type:
                    dic_counter[x] += 1
        return dic_counter[tag]
    
    else:
        return 0

In [15]:
def pos_features (df, speech_sents):
    
    """
    :function: add two new columns with two POS: adjectives and adverbs, and their counts per sentence.
    Two helper functions -- list_count, column_tag -- are needed 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: new DataFrame with two new features

    """
    
    
    df['pos'] = speech_sents.apply(lambda x: [(tag.text, tag.pos_) 
                                for tag in nlp(x)])
    
    df['pos'] = df['pos'].apply(lambda x: list_count(x))
    
    #extract features
    tags_set = []

    for lst in df['pos'].tolist():
        for dic in lst:
            for k in dic.keys():
                tags_set.append(k[1])
            
    tags_set = list(set(tags_set))

    for feature in tags_set:
        df['pos_' + feature] = df['pos'].apply(lambda x: column_tag(x, feature))
        
    # keeping only adverbs and adjectives and dropping other pos
    for feature in df.columns:
        if feature != 'pos_ADV' and feature != 'pos_ADJ' and 'pos' in feature:
            df = df.drop(feature, axis=1)
    
    return df

In [16]:
df = pos_features(df, df['Speech'])

### *Feature: Named Entity Recognition labels*

In [17]:
def ner_features(df, speech_sents):
    
    """
    :function: add several new columns with NER labels, and their counts per sentence.
    Two helper functions -- list_count, column_tag -- are needed 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: new DataFrame with new features for each NER label

    """
    
    df['ner'] = speech_sents.apply(lambda x: [(tag.text, tag.label_) 
                                for tag in nlp(x).ents])
    # count tags
    df['ner'] = df['ner'].apply(lambda x: list_count(x))
    
    # extract features
    tags_set = []

    for lst in df['ner'].tolist():
        for dic in lst:
            for k in dic.keys():
                tags_set.append(k[1])
            
    tags_set = list(set(tags_set))

    for feature in tags_set:
        df['ner_' + feature] = df['ner'].apply(lambda x: column_tag(x, feature))
        
    df = df.drop(['ner'], axis=1)
    
    return df 

In [18]:
df = ner_features(df, df['Speech'])

### *Feature: Verbs' tenses and modals*

In [21]:
def verbs_features (df, speech_sents):
    
    """
    :function: add several new columns with features for verb tenses and the presence of modal verbs, 
    and their counts per sentence.
    Two helper functions -- list_count, column_tag -- are needed 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: new DataFrame with features for each verb tense and for modal verbs

    """
    
    df['verb_tag'] = speech_sents.apply(lambda x: [(tag.text, tag.tag_) 
                                for tag in nlp(x)])
    
    df['verb_tag'] = df['verb_tag'].apply(lambda x: list_count(x))
    
    #extract features
    verbs_set = []

    for lst in df['verb_tag'].tolist():
        for dic in lst:
            for k in dic.keys():
                verbs_set.append(k[1])
            
    verbs_set = list(set(verbs_set))

    for feature in verbs_set:
        df['verb_tag_' + feature] = df['verb_tag'].apply(lambda x: column_tag(x, feature))
    
    #out of all detailed POS tags, keeping only verbs-related ones 
    for f in df.columns:
        if f != 'verb_tag_VB' and f != 'verb_tag_VBZ' and f != 'verb_tag_VBP' and f != 'verb_tag_VBD' and f != 'verb_tag_VBN' and f != 'verb_tag_VBG' and f != 'verb_tag_MD' and 'verb_tag' in f:
            df = df.drop(f, axis=1)
    
    
    return df

In [22]:
df = verbs_features(df, df['Speech'])

### *Feature: Syntactic features (the number of productions, the number of Verbal Phrases groups, the depth of a sentence tree)*

In [23]:
def syntactic_features(df, speech_sents):
        
    """
    :function: add syntactic features -- 1) the number of productions, 2) the number of VP groups per sentence, 
    and 3) the depth of a sentence tree 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: new DataFrame with three syntactic features 

    """
  
    a, b , c, d, e = [], [], [], [], []
    for x, y in enumerate(speech_sents):
        tagged = pos_tag(word_tokenize(y))
        chunker = RegexpParser(r"""
            NBAR:
            {<NN.*|JJ>*<NN.*>}  
            VP:
            {<V.*>}  
            NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  
        """)
    
        a.append(chunker.parse(tagged))
        b.append(len(chunker.parse(tagged).productions()))
        e.append(chunker.parse(tagged).productions())
        c.append(chunker.parse(tagged).height())

    df.loc[:, 'Speech_parsed'] = a
    df.loc[:, 'Productions_count'] = b
    df.loc[:, 'Tree_depth'] = c
  

    for i in e:
        vp = []
        for u in i:
            if str(u).startswith('VP'):
                vp.append(u)
        d.append(len(vp))
  
    df.loc[:, 'VP_count'] = d
    
    df = df.drop(['Speech_parsed'], axis=1)
    
    return df

In [24]:
df = syntactic_features(df, df['Speech'])

/Users/mariap/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:517: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.array(val, copy=False)
/Users/mariap/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


### *Feature: Sentiment of a sentence*

In [25]:
def add_sentiment (df, speech_sents): 
    
    
    """ 
    :function: add a feature with a sentiment score for each sentence 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: DataFrame with a new feature Sentiment
    
    """
    
    analyzer = SentimentIntensityAnalyzer()

    senti = []
    
    for sent in speech_sents:
        vs = analyzer.polarity_scores(sent)
        senti.append([list(vs.values())[3]])
    
    senti_arr = np.array(senti)
    df['Sentiment'] = senti_arr
    
    return df 

In [26]:
df = add_sentiment(df, df['Speech'])

### *Feature: Discourse connectives*

In [27]:
def add_connectives (df, speech_sents):

    """ 
    :function: add a boolean feature based on presence/absence of a claim connective from the pre-defined list 
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: DataFrame with a new feature Claim_Connective
    
    """
    
    connectives = ['so that', 'as a result', 'therefore', 'thus', 'thereby', 'in the end', 'hence', 'accordingly', 'in this way', 'because', 'now that', 'insofar as', 'given that', 'in response to', 'consequently', 'as a consequence']
    lst = []
    
    for sent in speech_sents:
        if any(w in sent for w in connectives):
            lst.append(1)
        else:
            lst.append(0)
    df['Claim_Connective'] = lst
    
    return df

In [28]:
df = add_connectives(df, df['Speech'])

### *Feature: Personal pronouns*

In [29]:
def add_personal(df, speech_sents):
    """
    :function: add two boolean features based on the presence/absence of any pronoun from two given lists.
    :input: df: entire DataFrame
            speech_sents: Series of sentences in DataFrame
    :return: df: DataFrame with two new features Pronoun_Singular and Pronoun_Plural

    """

    singular = [' i ', ' me ', ' my ', ' myself ', ' mine ']
    plural = [' we ', ' our ', ' ours ', ' ourselves ']
    lst_sing = []
    lst_plur = []

    for sent in speech_sents:
        if any(w in sent for w in singular):
            lst_sing.append(1)
        else:
            lst_sing.append(0)
    df['Pronoun_Singular'] = lst_sing

    for sent in speech_sents:
        if any(w in sent for w in plural):
            lst_plur.append(1)
        else:
            lst_plur.append(0)
    df['Pronoun_Plural'] = lst_plur

    return df

In [30]:
df = add_personal(df, df['Speech'])

### Tf-idf uni-, bi- and tri-grams and training preparations

**Splitting the data into three sets. Our sets will be identical to those of authors**

In [32]:
df_train = df[df['Set'] == 'TRAIN']
df_val = df[df['Set'] == 'VALIDATION']
df_test = df[df['Set'] == 'TEST']

In [33]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(10464, 37)
(5241, 37)
(6575, 37)


`(10464, 37)
(5241, 37)
(6575, 37)`

In [34]:
df_train = df_train.drop(['Set'], axis=1)
df_val = df_val.drop(['Set'], axis=1)
df_test = df_test.drop(['Set'], axis=1)

**Separating features set and target variable set**

In [35]:
X_train = df_train.drop(['Annotation'], axis=1)
y_train = df_train.Annotation

X_val = df_val.drop(['Annotation'], axis=1)
y_val = df_val.Annotation

X_test = df_test.drop(['Annotation'], axis=1)
y_test = df_test.Annotation

**Initializing tf-idf feature matrix. Fitting and transforming sentences on a train set and only transforming on a validation and test sets. Here we are using unigrams as well as bigrams and trigrams, set by the parameter `ngram_range`. Following the authors' practice, we will be using only the top occuring unigrams from the vocabulary, 10.000 n-grams**

In [36]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3))

#tf-idf
train_vecs =  vectorizer.fit_transform(X_train['Speech'])
val_tfidf = vectorizer.transform(X_val['Speech'])
test_vecs = vectorizer.transform(X_test['Speech'])

**Transformations of tf-idf matrix for further concatenation with other features**

In [37]:
names = vectorizer.get_feature_names()
dense = train_vecs.todense()
denselist = dense.tolist()
fe = pd.DataFrame(denselist, columns = names)

In [38]:
X_train = X_train.drop(['Speech'], axis=1)

In [39]:
train_features = np.hstack([X_train, fe])

In [42]:
#testing preparations
names = vectorizer.get_feature_names()
dense = test_vecs.todense()
denselist = dense.tolist()
fe = pd.DataFrame(denselist, columns = names)

In [43]:
X_test = X_test.drop(['Speech'], axis=1)

In [44]:
test_features = np.hstack([X_test, fe])

### Part 2: Replication

**First of all, repeating the authors' setting. Kernel is `rbf`, penalty parameter `C=10`**

In [45]:
svm = SVC(kernel='rbf', C=10, random_state=42)
svm.fit(train_features, y_train)

SVC(C=10, random_state=42)

In [46]:
y_pred_test_svm = svm.predict(test_features)

**class 1 stands for CLAIM, class 0 stands for PREMISE**

In [47]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred_test_svm, target_names=target_names, digits=3))

              precision    recall  f1-score   support

     class 0      0.694     0.458     0.552      3214
     class 1      0.609     0.806     0.694      3361

    accuracy                          0.636      6575
   macro avg      0.651     0.632     0.623      6575
weighted avg      0.650     0.636     0.624      6575



                      precision recall  f1-score   support

     class 0          0.694     0.458     0.552      3214
     class 1          0.609     0.806     0.694      3361

    accuracy                              0.636      6575
    macro avg         0.651     0.632     0.623      6575
    weighted avg      0.650     0.636     0.624      6575

### Part 3: Hyperparameter tuning

**Our results are close to authors', but not as high. We perform hyperparameter tuning to improve the scores.**

In [48]:
#validation set preparation
names = vectorizer.get_feature_names()
dense = val_tfidf.todense()
denselist = dense.tolist()
fe = pd.DataFrame(denselist, columns = names)

In [49]:
X_val = X_val.drop(['Speech'], axis=1)

In [50]:
val_features = np.hstack([X_val, fe])

**First, initializing the parameters grid. We are tuning parameters `C` and `kernel`, because linear kernel can be potentially good as seen from task 1, same experimental setting**

In [51]:
param_grid = {'C': [1, 10], 'kernel': ['linear','rbf']}

In [52]:
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(val_features, y_val)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=1, kernel=linear ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................... C=1, kernel=linear, total= 4.6min
[CV] C=1, kernel=linear ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.6min remaining:    0.0s


[CV] ............................... C=1, kernel=linear, total= 4.6min
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total= 4.6min
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total= 4.7min
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total= 4.4min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 4.9min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 4.8min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total= 5.0min
[CV] C=1, kernel=rbf .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 104.0min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ['linear', 'rbf']}, verbose=2)

In [53]:
grid.best_params_

{'C': 10, 'kernel': 'rbf'}

`'C': 10, 'kernel': 'rbf'`

**Conclusion: Hyperparameter tuning is suggesting that the original parameter setting is the best. Upon the results with original parameters, we can conclude that our results are rather close to those of authors, with average F1-score for both classes being 0.624, the authors' - 0.651**